# 可编程的交易

## 脚本引擎的工作原理

脚本引擎实际上是一个堆栈机，从左至右的操作脚本指令。

示例：脚本[2 3 ADD 5 EQUAL]的堆栈过程：

In [1]:
from simchain.vm import Stack
# 创建一个栈实例
stack = Stack()
# 常量2入栈
stack.push(2)
stack

[2]

2019-06-12 01:05:42,758 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# 常量3入栈
stack.push(3)
stack

[2, 3]

In [3]:
# 栈顶和次栈顶元素出栈相加
res = stack.pop() + stack.pop()
res

5

In [4]:
# 结果入栈
stack.push(res)
stack

[5]

In [5]:
# 常量5入栈
stack.push(5)
stack

[5, 5]

In [6]:
# 判断栈顶元素和次栈顶元素是否相等
flag = stack.pop() == stack.pop()
flag

True

In [7]:
# 将结果入栈
stack.push(flag)
stack

[True]

## 堆栈机LittleMachine

目前支持的运算符指令  
- OP_ADD：栈顶元素和次栈顶元素出栈，相加结果入栈
- OP_MINUS：栈顶元素和次栈顶元素出栈，相减结果入栈
- OP_MUL：栈顶元素和次栈顶元素出栈，相乘结果入栈
- OP_EQ：栈顶元素和次栈顶元素出栈，比较是否相等，相等则继续执行，否则验证失败
- OP_EQUAL：栈顶元素和次栈顶元素出栈，比较是否相等，相等则将True入栈，否则将False入栈
- OP_CHECKSIG：栈顶元素和次栈顶元素出栈，调用签名验证函数对签名进行验证，通过将True入栈，否则将False入栈
- OP_ADDR：栈顶元素出栈，调用公钥生成地址函数进行地址转换，并将结果入栈
- OP_DUP：复制栈顶元素并入栈
- OP_NDUP：复制栈顶n个元素并依次入栈
- OP_CHECKMULSIG：将指定数量的签名和公钥依次出栈，调用签名函数验证每个签名，全部通过则将True入栈，否则将False入栈
- OP_MULHASH：将n个公钥入栈，依次组合后进行哈希算法，并将结果入栈

堆栈机实现的Python代码如下：

In [8]:
from simchain.ecc import convert_pubkey_to_addr, VerifyingKey, sha256d

class LittleMachine(object):

    def __init__(self):
        
        # 初始化栈
        self.stack = Stack()
        
        # 定义操作符指令与空方法的映射关系
        self._map = {
            "OP_ADD":          self.add,
            "OP_MINUS":        self.minus,
            "OP_MUL":          self.mul,
            "OP_EQ":           self.equal_check,
            "OP_EQUAL"    :    self.equal,
            "OP_CHECKSIG":     self.check_sig,
            "OP_ADDR":         self.calc_addr,
            "OP_DUP"      :    self.dup,
            "OP_NDUP"     :    self.ndup,
            "OP_CHECKMULSIG" : self.check_mulsig,
            "OP_MULHASH":      self.calc_mulhash,
            }

    # 设置脚本
    def set_script(self,script,message = b''):
        # 清空栈
        self.clear()
        # 为真，堆栈继续，否则返回
        self.result = True
        # 操作指令的指针
        self.pointer = 0
        # 签名明文
        self.message = message
        # 脚本
        self.script = script
        
    #将栈清空
    def clear(self):
        self.stack.clear()
        
    # 获取栈顶元首    
    def peek(self):
        return self.stack.peek()
    
    # 出栈
    def pop(self):
        return self.stack.pop()

    # 入栈
    def push(self,value):
        self.stack.push(value)

    # 执行指令    
    def evaluate(self,op):
        # 如果指令在_map映射中，则执行对应的方法
        # 比如指令为OP_DUP，则执行dup()方法
        if op in self._map:
            self._map[op]()
        
        # 如果指令为常量入栈
        elif isinstance(op,str) or\
             isinstance(op,bytes)or\
             isinstance(op,int) or\
             isinstance(op,bool):
            self.push(op)
        else:
            logger.info('Uknow opcode: '.format(op))

    # 加法方法
    def add(self):
        self.push(self.pop() + self.pop())
    
    # 减法方法
    def minus(self):
        last = self.pop()
        self.push(self.pop() - last)

    # 乘法方法
    def mul(self):
        self.push(self.pop() * self.pop())

    # 复制栈顶元素并入栈
    def dup(self):
        self.push(self.peek())

    # 复制n个元素入栈
    def ndup(self):
        # 栈顶元素出栈，得到要复制的元素个数
        n = self.pop()
        # 将栈顶的n个元素复制到栈顶
        for val in self.stack[-n:]:
            self.push(val)
        # 将数量n复制到栈顶
        self.push(n)

    # 判断栈顶和次栈顶元素是否相等，不相等则堆栈结束
    def equal_check(self):
        flag = self.pop() == self.pop()
        if not flag:
            self.result = False

    # 判断栈顶和次栈顶元素是否相等，并将结果入栈
    def equal(self):
        self.push(self.pop()==self.pop())

    # 计算多公钥哈希值    
    def calc_mulhash(self):
        # 栈顶元素出栈，得到计算哈希的公钥个数n
        n = self.pop()
        # 将公钥依次出栈，并存储到列表中
        pk_strs = [self.pop() for _ in range(n)]
        # 将n个公钥组合进行哈希算法
        s = b''
        for val in pk_strs[::-1]:
            s += val
        self.push(sha256d(s))
        
    # 验证签名
    def check_sig(self):
        # 将栈顶元素出栈，并命名为pk_str,表示公钥字节串
        pk_str = self.pop()
        # 将次栈顶元素出栈，并命名为sig，表示签名
        sig = self.pop()
        # 用公钥字节串创建公钥对象
        verifying_key = VerifyingKey.from_bytes(pk_str)
        
        # 尝试用公钥对象验证签名
        try:
            flag = verifying_key.verify(sig,self.message)
        except Exception:
            flag = False
        self.push(flag)

    # 验证多重签名
    def check_mulsig(self):
        # 栈顶元素出栈，得到公钥数量n
        n = self.pop()
        # 将n个公钥分别出栈，并存储在列表pk_strs中
        pk_strs = [self.pop() for _ in range(n)]
        # 栈顶元素出栈，得到签名数量m
        m = self.pop()
        # 将m个签名分别出栈，并存储在列表sigs中
        sigs = [self.pop() for _ in range(m)]
        # 获取后m个公钥
        pk_strs = pk_strs[-m:]
        # 每个签名进行验证
        for i in range(m):
            verifying_key = VerifyingKey.from_bytes(pk_strs[i])
            try:
                flag = verifying_key.verify(sigs[i],self.message)
            except Exception:
                flag = False
            if not flag:
                falg = False
                break
        self.push(flag)
        
    # 计算地址
    def calc_addr(self):
        # 将栈顶元素出栈，并命名为pk_str，表示公钥字节串
        pk_str = self.pop()
        # 将公钥字节串转换为地址并入栈
        self.push(convert_pubkey_to_addr(pk_str))
    
    #运行脚本
    def run(self):
        # 如果操作指令指针大于等于脚本的长度，则退出循环
        while (self.pointer < len(self.script)):
            # 获取脚本指令
            op = self.script[self.pointer]
            # 操作指令指针加1
            self.pointer += 1
            # 执行操作指令
            self.evaluate(op)
        
        # 如果出现堆栈结束的情况，则返回为False
        if not self.result:
            return False
        # 否则返回栈顶元素
        else:
            return self.peek()

### LittleMachine的使用

In [9]:
from simchain.vm import LittleMachine
# 定义脚本
script = [2, 3, 'OP_ADD', 5, 'OP_EQUAL']
machine = LittleMachine()
machine.set_script(script)
machine.run()

True

In [10]:
script = [1, 'I', 'U', 'OP_DUP']
machine.set_script(script)
machine.run()

'U'

In [11]:
# 复制'U'
machine.stack

[1, 'I', 'U', 'U']

In [12]:
script = [1, 'I', 'U', 2, 'OP_NDUP']
machine.set_script(script)
machine.run()

2

In [13]:
# 复制'I', 'U'
machine.stack

[1, 'I', 'U', 'I', 'U', 2]

In [14]:
script = [1, b'I', b'U', 'OP_ADDR']
machine.set_script(script)
machine.run()

'15Kx26NBHA2521LZWYe9GHcgPa9PMKvPGj'

In [15]:
machine.stack

[1, b'I', '15Kx26NBHA2521LZWYe9GHcgPa9PMKvPGj']

In [16]:
script = [1, b'I', b'U', 2, 'OP_MULHASH']
machine.set_script(script)
machine.run()

'732a89979416a90e1a28f20a0ca9aa453936b37847a76a7ee769e1e4f343daef'

In [17]:
machine.stack

[1, '732a89979416a90e1a28f20a0ca9aa453936b37847a76a7ee769e1e4f343daef']

还可以对单个输入单元进行验证

In [18]:
from simchain import Pointer, Vin, Vout, Tx
from simchain.ecc import SigningKey, convert_pubkey_to_addr, VerifyingKey

# 假设这是张三的私钥
k = 3457534
# 用整数创建一个私钥对象
sk = SigningKey.from_number(k)
# 获取该私钥对应的公钥
pk = sk.get_verifying_key()
# 将公钥转换为对应的地址
addr = convert_pubkey_to_addr(pk.to_bytes())
# 用创币交易给张三发100的钱
coinbase = Tx.create_coinbase(addr, 100)
# 获取张三创币交易中的UTXO的定位指针
pointer = Pointer(coinbase.id , 0)
# 张三选择一条签名明文
message = b'I love blockchain'
# 用该明文签名
sig = sk.sign(message)
# 创建一个输入段元
vin = Vin(pointer, sig, pk.to_bytes())
# 输入单元的解锁脚本
vin.sig_script

b'\xa7\x0by\xd8,a\xcea\xd9^\xde \x16\xf3\xf7\x9dO`f(\xa1\x84\x9e\xef\xd3\xa5\xeb\xcc\x17\x9c-\xffw\x99=\xc7\xde\xf8\x8e\xf2\xb2\xabry\xd9C\xe2\xb0:h\xa9.!\x9e\xcc\x80\x8b\x87\xf7\xa2\xf2\xcek#(\xb4Dc\xfe\x11]fd\x9e\x14%\xa2\n\xb6L\xfe\xeb\x95\x90\xc2z!\xee\xedK\xc9\x1b+\xe0\x00\xfb\xafF?\x82[R\x8d\x1aI\xff\xc9\xf5\xdc\xb7|0|I\x05=\x064\x04*\xd8a\x93\xd2\xbd\x12|"'

In [19]:
# 解锁脚本长度为128
len(vin.sig_script)

128

In [20]:
# 私钥字节串长度为32
len(sk.to_bytes())

32

In [21]:
# 公钥字节串长度为64
len(pk.to_bytes())

64

In [22]:
# 签名长度为64
len(sig)

64

In [23]:
# 获取vin中使用的UTXO
vout = coinbase.tx_out[0]
# 获取锁定脚本
vout.pubkey_script

'OP_DUP OP_ADDR 1AiTFmgGc1uP5Q28RLFBiP4nH22UgPhkks OP_EQ OP_CHECKSIG'

In [24]:
# 张三的地址
addr

'1AiTFmgGc1uP5Q28RLFBiP4nH22UgPhkks'

In [25]:
# 解锁脚本解码
sig_script = [vin.sig_script[:64], vin.sig_script[64:]]
sig_script

[b'\xa7\x0by\xd8,a\xcea\xd9^\xde \x16\xf3\xf7\x9dO`f(\xa1\x84\x9e\xef\xd3\xa5\xeb\xcc\x17\x9c-\xffw\x99=\xc7\xde\xf8\x8e\xf2\xb2\xabry\xd9C\xe2\xb0:h\xa9.!\x9e\xcc\x80\x8b\x87\xf7\xa2\xf2\xcek#',
 b'(\xb4Dc\xfe\x11]fd\x9e\x14%\xa2\n\xb6L\xfe\xeb\x95\x90\xc2z!\xee\xedK\xc9\x1b+\xe0\x00\xfb\xafF?\x82[R\x8d\x1aI\xff\xc9\xf5\xdc\xb7|0|I\x05=\x064\x04*\xd8a\x93\xd2\xbd\x12|"']

In [26]:
# 将锁定脚本存储到列表
pubkey_script = vout.pubkey_script.split(' ')
pubkey_script

['OP_DUP',
 'OP_ADDR',
 '1AiTFmgGc1uP5Q28RLFBiP4nH22UgPhkks',
 'OP_EQ',
 'OP_CHECKSIG']

In [27]:
script = sig_script + pubkey_script
script

[b'\xa7\x0by\xd8,a\xcea\xd9^\xde \x16\xf3\xf7\x9dO`f(\xa1\x84\x9e\xef\xd3\xa5\xeb\xcc\x17\x9c-\xffw\x99=\xc7\xde\xf8\x8e\xf2\xb2\xabry\xd9C\xe2\xb0:h\xa9.!\x9e\xcc\x80\x8b\x87\xf7\xa2\xf2\xcek#',
 b'(\xb4Dc\xfe\x11]fd\x9e\x14%\xa2\n\xb6L\xfe\xeb\x95\x90\xc2z!\xee\xedK\xc9\x1b+\xe0\x00\xfb\xafF?\x82[R\x8d\x1aI\xff\xc9\xf5\xdc\xb7|0|I\x05=\x064\x04*\xd8a\x93\xd2\xbd\x12|"',
 'OP_DUP',
 'OP_ADDR',
 '1AiTFmgGc1uP5Q28RLFBiP4nH22UgPhkks',
 'OP_EQ',
 'OP_CHECKSIG']

In [28]:
machine = LittleMachine()
machine.set_script(sig_script + pubkey_script, message)
machine.run()

True

可对交易的每一个输入单元进行验证

In [29]:
# 节点对每个Vin进行验证
def verify_signature(peer,vin,utxo,tx_out):
    # 获取解锁脚本和锁定脚本的组合
    script = check_script_for_vin(vin,utxo,peer.key_base_len)
    # 判空
    if not script:
        return False
    
    # 选择验证签名的明文
    string = str(vin.to_spend) + str(vin.pubkey) + str(tx_out)
    message = build_message(string)
    # 设置节点堆栈机的脚本和签名明文
    peer.machine.set_script(script,message)
    # 返回结果
    return peer.machine.run()

# 检查脚本格式并组合
def check_script_for_vin(vin,utxo,baselen):
    # 从输入单元中获取解锁脚本，从UTXO中获取锁定脚本
    sig_script,pubkey_script = vin.sig_script,utxo.pubkey_script
    
    # 基本编码长度的2倍和4倍数，基本编码长度是1个整数的编码长度，如果私钥的长度在椭圆曲线签名中，签名是2个整数，所以长度是2倍的基本编码长度
    # 公钥是一个点，也是2个整数，所以也是2倍的基本编码长度，而解锁甲苯包含签名和公钥，所以长度是4倍。
    double,fourfold = int(baselen*2),int(baselen*4)
    if len(sig_script) != fourfold:
        return False
    sig_scrip = [sig_script[:double],sig_script[double:]]
    try:
        pubkey_script = pubkey_script.split(' ')
    except Exception:
        return False

    return sig_scrip+pubkey_script

## 多重签名

用LittleMachine进行多重签名的示例

In [30]:
from simchain import SigningKey, sha256d

# A的私钥，整数
kA = 3453543
# B的私钥，整数
kB = 2349334
# 创建A、B的私钥对象
skA = SigningKey.from_number(kA)
skB = SigningKey.from_number(kB)
# A、B的公钥
pkA = skA.get_verifying_key()
pkB = skB.get_verifying_key()
# 签名明文
message = b'I love blockchain'
# A、B的签名
sigA = skA.sign(message)
sigB = skB.sign(message)
# 组合A和B的公钥并计算哈希值
Hash = sha256d(pkA.to_bytes() + pkB.to_bytes())
# 解锁脚本
sig_script = [sigA, sigB, 2, pkA.to_bytes(), pkB.to_bytes(), 2]
# UTXO中的锁定脚本
pubkey_script = ['OP_NDUP', 'OP_MULHASH', Hash, 'OP_EQ', 2, 'OP_CHECKMULSIG']
# 组合脚本
script = sig_script + pubkey_script

In [31]:
machine = LittleMachine()
machine.set_script(script, message)
machine.run()

True